<a href="https://colab.research.google.com/github/PeterHJY628/Llama3.2-Vision-Finetune/blob/master/DEFT_GaLore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Surgical LLM Agent Demo

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create output directories in Google Drive
import os
output_base_dir = '/content/drive/MyDrive/surgical_llm_demo'
model_dir = f'{output_base_dir}/models/r128'
data_dir = f'{output_base_dir}/datasets'
results_dir = f'{output_base_dir}/results'
metrics_dir = f'{output_base_dir}/metrics'

# Create directories if they don't exist
for directory in [output_base_dir, model_dir, data_dir, results_dir, metrics_dir]:
    os.makedirs(directory, exist_ok=True)

print(f"Created output directories in Google Drive at: {output_base_dir}")

# Install required packages
!pip install -q transformers==4.45.1 datasets==3.0.1 evaluate==0.4.3 gdown nltk torch==2.4.0 torchvision==0.19.0
!pip install -q huggingface_hub
!pip install -q bitsandbytes>=0.43.2
!pip install -q accelerate
!pip install rouge_score

# Download necessary NLTK data
import nltk
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)

# Download model weights using gdown (NOTE: Replace with your actual file ID)
print("Downloading model weights...")
MODEL_FILE_ID = "1l5tJ41cQa0M8z0UF8Az96DX1sZMAV0Yt"  # Replace with the actual file ID from Google Drive
#https://drive.google.com/drive/folders/1c6dLqsDnivVClRBdzfeZbd4HvLkIEfah?usp=drive_link
#https://drive.google.com/drive/folders/1l5tJ41cQa0M8z0UF8Az96DX1sZMAV0Yt?usp=sharing

# Check if model is already downloaded
if not os.path.exists(f"{model_dir}/config.json"):
    !gdown --folder {MODEL_FILE_ID} -O {model_dir}
    print(f"Model weights downloaded to {model_dir}")
else:
    print(f"Model weights already exist at {model_dir}")

# Download datasets
print("Downloading datasets...")
DATASET_FILE_IDS = {
    "Surgical-VQA_V.csv": "1rjv3PzKHqz5BjJn8anR2jSwNTPS2k0Ak",
    "Overlaying_V.csv": "1gFpt8kjoc0kzTBXXiRlYgwDC-HzlSGAr",
    "Segment-MRI_V.csv": "1rSJfPEqg24fhk4MybqpRw652orLfgojc",
    "Segment-Video_V.csv": "1lo0xEKcJgMPy0T0AXfRIjNbrSdJyrXkR",
    "Detect-Instrument_V.csv": "1A4c5ieW6P_oqMnWRybl6NmtmTshsPX1n",
    "2model_V.csv": "1dl_81gH1o06ZYLn1FuL8J4INxq3cLqnu",
    "3model_V.csv": "1WwLigg0kjRHyxkOaSYc8V2M8GFK9qlc9"
}

# https://drive.google.com/file/d/1rjv3PzKHqz5BjJn8anR2jSwNTPS2k0Ak/view?usp=drive_link
# https://drive.google.com/file/d/1gFpt8kjoc0kzTBXXiRlYgwDC-HzlSGAr/view?usp=drive_link
# https://drive.google.com/file/d/1rSJfPEqg24fhk4MybqpRw652orLfgojc/view?usp=drive_link
# https://drive.google.com/file/d/1lo0xEKcJgMPy0T0AXfRIjNbrSdJyrXkR/view?usp=drive_link
# https://drive.google.com/file/d/1A4c5ieW6P_oqMnWRybl6NmtmTshsPX1n/view?usp=drive_link
# https://drive.google.com/file/d/1dl_81gH1o06ZYLn1FuL8J4INxq3cLqnu/view?usp=drive_link
# https://drive.google.com/file/d/1WwLigg0kjRHyxkOaSYc8V2M8GFK9qlc9/view?usp=drive_link

# Download each dataset
for filename, file_id in DATASET_FILE_IDS.items():
    if not os.path.exists(f"{data_dir}/{filename}"):
        !gdown {file_id} -O {data_dir}/{filename}
        print(f"Downloaded {filename}")
    else:
        print(f"{filename} already exists")

Mounted at /content/drive
Created output directories in Google Drive at: /content/drive/MyDrive/surgical_llm_demo
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━

# Define Fuctions

In [ ]:
# Import required libraries
import torch
import pandas as pd
import re
import evaluate
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm.notebook import tqdm
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

warnings.filterwarnings("ignore", category=UserWarning)

# ---------------------------- Utility Function ----------------------------
def generate_SM(que: str) -> str:
    return (
        "You are a surgical AI agent assisting in pituitary surgery. Your job is to handle surgeons' queries efficiently by choosing appropriate text-promptable AI models and generating corresponding prompts.\n"
        "Available models: Segment-Video, Segment-MRI, Track-Instrument, Surgical-VQA, Overlaying.\n"
        "Question: {que}\n"
        "- Use ONE model if query focuses on a single, simple aspect:\n"
        "Example (single-model):\n"
        "Model: Segment-Video\nPrompt: Segment the sella in the video.\n"
        "- Use MULTIPLE models if query requires several types of information:\n"
        "Example (multi-model):\n"
        "Step1:\nModel: Segment-MRI\nPrompt: Segment the pituitary tumor from MRI.\n"
        "Step2:\nModel: Segment-Video\nPrompt: Segment the sella in the video.\n"
        "Now, follow the same format to answer the provided question—no extra text, labels, or formatting."
    ).format(que=que)

def generate_answer(question, model, tokenizer):
    model.eval()
    question = generate_SM(question)
    input_text = f"Query:\n{question}\nResponse:\n"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Response:\n")[-1].strip()
    return answer




#Inference

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model using BitsAndBytes for efficient loading
print("Loading model...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

try:
    # Load model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(
        model_dir,
        quantization_config=bnb_config,
        device_map="auto"
    )

    tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False)
    tokenizer.pad_token = tokenizer.eos_token

    print("Model loaded successfully!")



except Exception as e:
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

Loading model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded successfully!


In [ ]:
# Define Input question
# print("\nDefine the sample query question here:")
# Define the sample query question here:
question = "Segment the sella in the video?"

answer = generate_answer(question, model, tokenizer)
print("The answer generated by agent is:\n ")
print(answer)



The answer generated by agent is:
 
Model: Segment-Video
Prompt: Segment the sella in the video.
